## Requirements 

In [ ]:
!pip3 install Cython #speeds up the loading of large ontologies (optional)
!pip3 install owlready2

## Load an Ontology

The sample ontology can be found [here](http://www.lesfleursdunormal.fr/static/_downloads/pizza_onto.owl)

In [ ]:
from owlready2 import *

In [ ]:
onto = get_ontology("http://www.lesfleursdunormal.fr/static/_downloads/pizza_onto.owl").load()

## Accessing the content of an ontology

In [ ]:
list(onto.classes())

In [ ]:
list(onto.individuals())

In [ ]:
list(onto.object_properties())

## Simple Queries

In [ ]:
# searching for all entities with an IRI (Resource Identifier or ID) ending with ‘Topping’
onto.search(iri = "*Topping") 

In [ ]:
#searches for all individuals that are related to another one with the ‘has_topping’ relation
onto.search(has_topping = "*") # '*' acts as a wildcard for any object

In [ ]:
#create new instances / individuals
my_pizza = onto.Pizza("my_perfect_pizza", has_topping=[ onto.CheeseTopping(), onto.MeatTopping()])

In [ ]:
onto.search(has_topping = "*")

In [ ]:
my_pizza.has_topping

In [ ]:
#classes and individuals can be used as values within search
onto.search(is_a = onto.Pizza, has_topping = onto.search(is_a = onto.CheeseTopping)) 

In [ ]:
# iterate through all Instances of a Class
for i in onto.Pizza.instances(): print(i)

In [ ]:
# The list of properties that exist for a given individual 
my_pizza.get_properties()

## Ontology Classes and Properties

In [ ]:
# Create new class in the ontology
with onto:
    
    # New Ontology objects can be created by sublcassing the Thing class
    class Restaurant(Thing): # Ontology Object
        pass
    
    class Restaurant2(): # Normal Python Class
        pass
    
    # New Ontology properties can be created by sublcassing the ObjectProperty class
    class servesPizza(ObjectProperty):
        domain    = [Restaurant] #the class of instance subjects of the property
        range     = [onto.Pizza] #the class of instance objects of the property

    # Subclasses can be created by inheriting an ontology class
    class PizzawithToppings(onto.Pizza):
        # equivalent_to behaves like is_a and allows for adding more specifications
        equivalent_to = [
            onto.Pizza & onto.has_topping.min(1, onto.Topping) #other restrictions include: some, only, max, exactly
        ]
        def who(self): print("I'm a Pizza with Toppings!")

    class NonVegetarianPizza(PizzawithToppings):
        equivalent_to = [
            PizzawithToppings & ( onto.has_topping.some(onto.MeatTopping) | onto.has_topping.some(onto.FishTopping) )
        ]
        def who(self): print("I'm a Non Vegetarian Pizza!")
    

In [ ]:
fishPizza = NonVegetarianPizza("fishPizza", has_topping=[ onto.FishTopping()])
pizzaHut = Restaurant("PizzaHut", servesPizza=[fishPizza])

In [ ]:
Restaurant.instances()

In [ ]:
pizzaHut.servesPizza

## Reasoning

In [ ]:
my_pizza.__class__

In [ ]:
# Execute HermiT reasoner and reclassify ontology statements based on ontology axioms
# This creates new facts (triplets) and adds then into the ontology
# Under windows, you may need to configure the location of the Java interpreter
# owlready2.JAVA_EXE = "C:\\path\\to\\java.exe"
with onto:
    sync_reasoner()

In [ ]:
my_pizza.__class__

In [ ]:
my_pizza.who()

## Reasoning Inconsistent Ontology

* In case of inconsistent ontology, an OwlReadyInconsistentOntologyError is raised.


In [ ]:
with onto:
    class InconsistentPizza(PizzawithToppings):
        equivalent_to = [
            PizzawithToppings & ( onto.has_topping.some(onto.Pizza) )
        ]
       

* Inconcistent classes may occur without making the entire ontology inconsistent, as long as these classes have no individuals.
* Inconsistent classes are inferred as equivalent to Nothing (empty set).

In [ ]:
my_pizza2 = InconsistentPizza("Pizza2", has_topping=[onto.Pizza])
print(my_pizza2.__class__)
print(my_pizza2.has_topping)

* Apply reasoner again and notice the produced error OwlReadyInconsistentOntologyError.

### Resolve Inconsistency

* Ontology editors like [Protégé](https://protege.stanford.edu/) can provide reasons for the inconsistency.
* Inconsistencies can also be found by searching for instances of inconsistent classes and removing them.

In [ ]:
# One way to check if a class is inconsistent (after reasoning)
if Nothing in InconsistentPizza.equivalent_to:
    print("Class is inconsistent!")

In [ ]:
# List all inconsistent classes in the ontology
inc_classes = list(default_world.inconsistent_classes())
print(inc_classes)

In [ ]:
# Find instances of the inconsistent class
for inc_class in inc_classes:
    print("Removing instances of", inc_class)
    for instance in onto.get_instances_of(inc_class):
        print("\tRemoving", instance)
        destroy_entity(instance) #remove it from the ontology to enable the reasoner to run

* Apply reasoner again and notice the results.

## The Nothing Class

* This class exists in the ontology as concept only without any instances.
* To keep the ontology sound, this class must not be instantiated.

In [ ]:
Nothing.__class__

In [ ]:
# It should not be possible to create instances of the Nothing class!
with onto:
    nothing = Nothing("Empty")

In [ ]:
list(onto.individuals())

## Save the Ontology

In [ ]:
#save the updated ontology as RDF/XML
onto.save(file = "new_pizza_onto.owl", format = "rdfxml") #OWL/XML is not yet supported for writing.